In [ ]:
# ====================================================
# Phase 1: Data Preparation
# Project: Predictive Analytics for Supply Chain Optimization
# ====================================================

# --- 1/1. Import Required Libraries ---
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    accuracy_score, classification_report, confusion_matrix
)


In [ ]:
# --- 1/2. Import data ---
try:
  df = pd.read_csv(r"C:\Users\BOB\Documents\Data Analysis\.data\dynamic_supply_chain_logistics_dataset.csv", encoding='utf-8')
  cities_df = pd.read_csv(r"C:\Users\BOB\Documents\Data Analysis\.data\uscities.csv")
except UnicodeDecodeError:
  df = pd.read_csv(r"C:\Users\BOB\Documents\Data Analysis\.data\dynamic_supply_chain_logistics_dataset.csv", encoding='latin-1')
  cities_df = pd.read_csv(r"C:\Users\BOB\Documents\Data Analysis\.data\uscities.csv")
print("Dataset loaded successfully!")


# Preview
# df.head()
# df_city.head()


# Show list of columns
# print(df_city.columns)

In [ ]:
# --- 1/3. Basic Data Overview ---

print("Shape of dataset:", df.shape)
# print("\nColumns:\n", df.columns.tolist())
print("\nMissing values summary:\n", df.isna().sum())
print("\nData types:\n", df.dtypes)

In [ ]:
# --- 1/5. Handle Missing Values with NumPy for data integrity---
# Identify numeric columns, but exclude 'timestamp'
num_cols = df.select_dtypes(include=np.number).columns.tolist()
num_cols = [col for col in num_cols if col != 'timestamp']

# Identify categorical columns
# cat_cols = df.select_dtypes(exclude=np.number).columns.tolist()

# Impute numeric columns with median
for col in num_cols:
    median_val = np.nanmedian(df[col])
    df[col] = df[col].fillna(median_val)

# If 'timestamp' is datetime and has missing values, optionally impute separately
if df['timestamp'].isna().any():
    # Replace NaT with some default, e.g., first date
    df['timestamp'] = df['timestamp'].fillna(df['timestamp'].min())


# print(df['timestamp'].head())


In [ ]:
# --- 1/7 Nearest City Assignment Using KDTree ---
from scipy.spatial import cKDTree
import numpy as np
from geopy.distance import geodesic

# Example: your cities DataFrame
# cities = pd.read_csv("uscities.csv")
# Keep only necessary columns
cities = cities_df[['city', 'state_id', 'lat', 'lng']]

# Build KDTree
city_coords = cities_df[['lat', 'lng']].values
tree = cKDTree(city_coords)

# Vehicle coordinates
vehicle_coords = df[['vehicle_gps_latitude', 'vehicle_gps_longitude']].values

# Query nearest city (k=1 for nearest)
distances, idx = tree.query(vehicle_coords, k=1)

# Optional: compute approximate distance in km (using Haversine)
def haversine(lat1, lon1, lat2, lon2):
    # Radius of earth in km
    R = 6371.0
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda/2)**2
    return R * 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

# Compute distances for all rows
vehicle_lat = df['vehicle_gps_latitude'].values
vehicle_lon = df['vehicle_gps_longitude'].values
city_lat = cities_df.iloc[idx]['lat'].values
city_lon = cities_df.iloc[idx]['lng'].values

dist_km = haversine(vehicle_lat, vehicle_lon, city_lat, city_lon)

# Set threshold, e.g., 50 km
threshold_km = 50

# Assign vehicle_city only if within threshold
# Pre-build city + state
cities_df["city_full"] = cities_df["city"].astype(str) + ", " + cities_df["state_id"].astype(str)

# Assign nearest city only when it is within threshold
df["vehicle_city"] = np.where(
    dist_km <= threshold_km,
    cities_df.iloc[idx]["city_full"].values,
    np.nan
)
# Remove rows with any NaN values
# df_clean = df.dropna()
df = df.dropna()

# Check the result
# print(df_clean.shape)  # number of rows and columns after dropping
# print(df['vehicle_city'])


In [ ]:
# 1/8 Add a unique ID for each row
df['row_id'] = range(1, len(df) + 1)

# Check
# print(df[['row_id', 'vehicle_gps_latitude', 'vehicle_gps_longitude']].head())


In [ ]:
# --- 1/9 Feature Engineering ---

## 5.1 Create derived features
# Example: combine GPS into one column
df['vehicle_location'] = df['vehicle_gps_latitude'].astype(str) + ',' + df['vehicle_gps_longitude'].astype(str)

# Example: compute delay difference in minutes
df['eta_variation_minutes'] = df['eta_variation_hours'] * 60

# Example: categorize traffic level
df['traffic_category'] = pd.cut(
    df['traffic_congestion_level'],
    bins=[0, 3, 6, 10],
    labels=['Low', 'Medium', 'High']
)

# Example: flag if weather severity exceeds threshold
df['severe_weather_flag'] = np.where(df['weather_condition_severity'] > 0.7, 1, 0)

# ===========================================================
print("\n Feature engineering complete. New columns added:", 
      [c for c in df.columns if 'vehicle_location' in c or 'eta_variation_minutes' in c or 'traffic_category' in c or 'severe_weather_flag' in c])

In [ ]:
# --- 1/10. Clustering ---
from sklearn.cluster import KMeans
import numpy as np

# --- Temporal features ---
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['eta_minus_actual_hours'] = df['eta_variation_hours'] - df['delivery_time_deviation']
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['month'] = df['timestamp'].dt.month
df = df.sort_values(['row_id', 'timestamp'])

df['avg_route_congestion'] = (
    df.groupby('route_risk_level')['traffic_congestion_level']
      .transform(lambda x: x.rolling(5).mean())
)

# --- Clustering ---
coords = df[['vehicle_gps_latitude', 'vehicle_gps_longitude']]
kmeans = KMeans(n_clusters=5, random_state=42)
df['route_cluster'] = kmeans.fit_predict(coords)

# --- Cluster statistics ---
cluster_stats = (
    df.groupby('route_cluster')
      .agg(
          avg_delay=('delivery_time_deviation', 'mean'),
          avg_risk=('route_risk_level', 'mean'),
          avg_congestion=('traffic_congestion_level', 'mean'),
          lat=('vehicle_gps_latitude', 'mean'),
          lon=('vehicle_gps_longitude', 'mean')
      )
      .reset_index()
)

# Determine geographic labels
lat_median = cluster_stats['lat'].median()
lon_median = cluster_stats['lon'].median()

def geo_label(row):
    parts = []
    # North / South
    parts.append("Northern" if row['lat'] > lat_median else "Southern")
    # East / West
    parts.append("Eastern" if row['lon'] > lon_median else "Western")
    return " ".join(parts)

cluster_stats['geo'] = cluster_stats.apply(geo_label, axis=1)

# Categorize delay
def delay_label(v):
    return "High-delay" if v > cluster_stats['avg_delay'].median() else "Stable-delivery"

# Categorize risk
def risk_label(v):
    return "High-risk" if v > cluster_stats['avg_risk'].median() else "Low-risk"

# Congestion
def congestion_label(v):
    return "Congested" if v > cluster_stats['avg_congestion'].median() else "Free-flow"

cluster_stats['delay_text'] = cluster_stats['avg_delay'].apply(delay_label)
cluster_stats['risk_text'] = cluster_stats['avg_risk'].apply(risk_label)
cluster_stats['cong_text'] = cluster_stats['avg_congestion'].apply(congestion_label)

# FINAL NAME
cluster_stats['cluster_name'] = (
    cluster_stats['geo'] + " — " +
    cluster_stats['delay_text'] + ", " +
    cluster_stats['risk_text'] + ", " +
    cluster_stats['cong_text']
)

# Map into df
name_map = cluster_stats.set_index('route_cluster')['cluster_name'].to_dict()
df['cluster_desc'] = df['route_cluster'].map(name_map)

cluster_stats


In [ ]:
# --- 1/11. Categorical columns ---
# Identify which columns are categorical
cat_cols = df.select_dtypes(include=['object', 'category']).columns

# cat_cols = df.select_dtypes(exclude=np.number).columns
print("Categorical columns:", list(cat_cols))


In [ ]:
# --- 1/12. Categorical columns ---
# Categorical columns → replace NaN with most frequent value (mode)
for col in cat_cols:
    mode_val = df[col].mode()[0] if not df[col].mode().empty else 'Unknown'
    df[col] = df[col].fillna(mode_val)

print("\n Missing values handled with NumPy (median/mode).")


In [ ]:
# --- 1/14. Final Dataset Summary ---
print("\nFinal dataset shape:", df.shape)
print("\nSample data:")
display(df.head())
# print(df.shipping_costs)

In [ ]:
# ====================================================
# Phase 2: Exploratory Data Analysis (EDA)
# ====================================================

# --- 2/1. Import Required Libraries ---

import seaborn as sns

# display settings
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid", context="notebook")

# Assume df is already loaded and cleaned from Phase 1
print(" DataFrame available with shape:", df.shape)

In [ ]:
# --- 2/2. Route clusters visualization---
# Ensure necessary columns exist
required_cols = [
    'vehicle_gps_latitude',
    'vehicle_gps_longitude',
    'route_cluster',
    'delivery_time_deviation'
]

for col in required_cols:
    if col not in df.columns:
        raise ValueError(f"Missing column: {col}")

# Plot
plt.figure(figsize=(12, 8))

sns.scatterplot(
    data=df,
    x='vehicle_gps_longitude',
    y='vehicle_gps_latitude',
    hue='cluster_desc',      # ✔ Человеческие описания в легенде
    size='delivery_time_deviation',
    palette='tab10',
    sizes=(40, 300),
    alpha=0.8,
    legend="brief"
)

plt.title("Route Clusters Based on GPS Coordinates\n(Bubble Size = Delivery Time Deviation)", fontsize=16)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid(True, alpha=0.3)

plt.show()


In [ ]:
# --- 2/3. Prepare stat data ---
# Define the feature columns (independent variables)
features = [
    "fuel_consumption_rate", "traffic_congestion_level", "weather_condition_severity",
    "warehouse_inventory_level", "loading_unloading_time", "handling_equipment_availability",
    'route_cluster', "port_congestion_level", "shipping_costs", "supplier_reliability_score", ""
    "lead_time_days", "route_risk_level", "driver_behavior_score", "fatigue_monitoring_score"
]

# Define the target column (dependent variable)
# ===============================================
target = "delivery_time_deviation"
# ===============================================

# Keep only the selected columns and remove rows with missing data
stat_df = df[features + [target]].dropna()

# Check the dataset after cleaning
print(f"Data after filtering: {stat_df.shape[0]:,} rows and {stat_df.shape[1]} columns")

# Display summary statistics of the cleaned dataset
stat_df.describe().T


In [ ]:
    # --- 2.4 Scale Numerical Features ---
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()  # default range is 0 to 1
stat_df[features] = scaler.fit_transform(stat_df[features])
print(stat_df[features].head())


# Check result
print(stat_df[features].head())
print("\nCategorical encoding and scaling complete.")

In [ ]:
# --- 2/5. Data Correlation ---
from sklearn.feature_selection import mutual_info_regression
from sklearn.ensemble import RandomForestRegressor

# Assume stat_df has the same feature and target columns
# -----------------------------
# 1️ Spearman correlation (monotonic relationships)
spearman_corr = stat_df.corr(method='spearman')[target].drop(target).sort_values(ascending=False)
# ================================================================================================


plt.figure(figsize=(8,5))
sns.barplot(x=spearman_corr.values, y=spearman_corr.index, hue=spearman_corr.index, dodge=False, palette="viridis", legend=False)
plt.title("Spearman Correlation with Delivery Time Deviation (stat_df)")
plt.xlabel("Spearman Correlation")
plt.ylabel("Feature")
plt.show()

# -----------------------------
# 2️ Mutual Information (non-linear dependencies)
X_stat = stat_df[features]
y_stat = stat_df[target]
mi = mutual_info_regression(X_stat, y_stat, random_state=42)
mi_series = pd.Series(mi, index=features).sort_values(ascending=False)

plt.figure(figsize=(8,5))
plt.figure(figsize=(8, 5))
sns.barplot(
    x=mi_series.values,
    y=mi_series.index,
    hue=mi_series.index,   # required for palette
    dodge=False,
    palette="viridis",
    legend=False
)
plt.title("Mutual Information with Delivery Time Deviation (stat_df)")
plt.xlabel("Mutual Information")
plt.ylabel("Feature")
plt.show()

# -----------------------------
# 3️ Feature importance using Random Forest
rf = RandomForestRegressor(n_estimators=200, random_state=42)
rf.fit(X_stat, y_stat)
rf_importances = pd.Series(rf.feature_importances_, index=features).sort_values(ascending=False)

plt.figure(figsize=(8,5))
sns.barplot(x=rf_importances.values, y=rf_importances.index, hue=spearman_corr.index, dodge=False, palette="magma", legend=False)
# sns.barplot(x=rf_importances.values, y=rf_importances.index, palette="magma")
plt.title("Random Forest Feature Importance (stat_df)")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.show()

In [ ]:
# ====================================================
# 3️ DISTRIBUTION ANALYSIS AND ANOMALY DETECTION
# ====================================================
# --- 3/1. Plot distribution of key numeric variables---

key_vars = [
    'delivery_time_deviation',
    'fuel_consumption_rate',
    'traffic_congestion_level',
    'shipping_costs',
    'lead_time_days'
]

key_vars = [v for v in key_vars if v in df.columns]
print(key_vars)

for col in key_vars:
    plt.figure(figsize=(7, 4))
    sns.histplot(df[col], bins=30, kde=True)
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.show()

In [ ]:
# --- 3/2. Detect anomalies using IQR method for each key numeric column---

key_vars = [    
    'fuel_consumption_rate',
    'traffic_congestion_level',
    'shipping_costs',
    'lead_time_days'
]


# --- Boxplot for key numeric columns ---
plt.figure(figsize=(12, 6))
sns.boxplot(data=stat_df[key_vars], orient='h', palette='Set2')
plt.title("Boxplot of Key Numeric Variables (Outliers shown)")
plt.xlabel("Value")
plt.ylabel("Variables")
plt.show()


In [ ]:
# --- 3/3. Remove anomalies using different methods and replace the data in primary df---
import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler, StandardScaler

# --- Step 0: Define variables ---
skewed_vars = ['fuel_consumption_rate', 'lead_time_days']
robust_var = 'fuel_consumption_rate'
standard_vars = [v for v in stat_df.columns if v not in skewed_vars]

# --- Step 1: Log-transform skewed features ---
for col in skewed_vars:
    if col in stat_df.columns:
        stat_df[col] = np.log1p(stat_df[col])

# --- Step 2: Outlier treatment for fuel_consumption_rate ---
Q1 = stat_df[robust_var].quantile(0.25)
Q3 = stat_df[robust_var].quantile(0.75)
IQR = Q3 - Q1
lower_whisker = Q1 - 1.5 * IQR
upper_whisker = Q3 + 1.5 * IQR

# Option 1: Remove outliers
# stat_df = stat_df[(stat_df[robust_var] >= lower_whisker) & (stat_df[robust_var] <= upper_whisker)]

# Option 2: Cap outliers (recommended to keep dataset size)
stat_df[robust_var] = np.where(
    stat_df[robust_var] > upper_whisker, upper_whisker,
    np.where(stat_df[robust_var] < lower_whisker, lower_whisker, stat_df[robust_var])
)

# --- Step 3: Apply RobustScaler to fuel_consumption_rate ---
robust_scaler = RobustScaler()
stat_df[robust_var] = robust_scaler.fit_transform(stat_df[[robust_var]])

# --- Step 4: Apply StandardScaler to other features ---
if standard_vars:
    standard_scaler = StandardScaler()
    stat_df[standard_vars] = standard_scaler.fit_transform(stat_df[standard_vars])

# preserve df for machine learning
m_df = df.copy()


# --- Replace processed columns back into primary DataFrame, excluding the target ---
for col in stat_df.columns:
    # if col == "delivery_time_deviation":
    #     continue  # skip the target column
    
    if col in df.columns:
        df.loc[stat_df.index, col] = stat_df[col]


# --- Optional: Check the first few rows ---
# print(df.head())


In [ ]:
# --- 3/4 Boxplot for key numeric columns ---
plt.figure(figsize=(12, 6))
sns.boxplot(data=stat_df[key_vars], orient='h', palette='Set2')
plt.title("Boxplot of Key Numeric Variables (Outliers shown)")
plt.xlabel("Value")
plt.ylabel("Variables")
plt.show()

In [ ]:
# 3/5 Summary statistics by cluster_desc
summary_by_cluster = df.groupby('cluster_desc').agg(
    avg_delay=('delivery_time_deviation', 'mean'),
    avg_congestion=('traffic_congestion_level', 'mean'),
    avg_fuel=('fuel_consumption_rate', 'mean'),
    avg_shipping=('shipping_costs', 'mean')
).reset_index()

sns.set(style="whitegrid")

# Plot: Average delivery delay by cluster
plt.figure(figsize=(12, 6))
sns.barplot(
    x='avg_delay',
    y='cluster_desc',
    hue='cluster_desc',        # set hue same as y
    data=summary_by_cluster.sort_values('avg_delay', ascending=False),
    palette='Reds_r',
    dodge=False,               # avoid splitting bars
    legend=False               # hide the extra legend
)
plt.title("Average Delivery Delay by Cluster")
plt.xlabel("Average Delivery Time Deviation (hours)")
plt.ylabel("Cluster Description")
plt.tight_layout()
plt.show()


In [ ]:
# --- 3/6. Save Cleaned Dataset ---
df.to_csv('cleaned_logistics_data.csv', index=False)
print("\n Cleaned dataset saved as 'cleaned_logistics_data.csv'.")


In [ ]:
# ====================================================
# 4️ SEASONAL & REGIONAL DELAY TRENDS
# ====================================================
# --- 4/1. fuel consumption rate by month---

#correlation between fuel consuntion rate and temperature inside


# Convert timestamp to datetime if needed
if not np.issubdtype(df['timestamp'].dtype, np.datetime64):
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

# Extract time-based features
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['weekday'] = df['timestamp'].dt.day_name()

# --- 4.1 Seasonal trends (monthly average delay)
if 'delivery_time_deviation' in df.columns:
    monthly_delay = df.groupby('month')['delivery_time_deviation'].mean()
    plt.figure(figsize=(8, 4))
    sns.lineplot(x=monthly_delay.index, y=monthly_delay.values, marker='o')
    plt.title(" Average Delivery Time Deviation by Month")
    plt.xlabel("Month")
    plt.ylabel("Avg Delivery Deviation")
    plt.show()

In [ ]:
# --- 4.3 Weekday delay trends

if 'delivery_time_deviation' in df.columns:
# Define correct order
    weekday_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

    # Convert to categorical with order
    df['weekday'] = pd.Categorical(df['weekday'], categories=weekday_order, ordered=True)

    # Group and sort
    weekday_delay = df.groupby('weekday', observed=False)['delivery_time_deviation'].mean().sort_index()

    # print(weekday_delay)

    # weekday_delay = df.groupby('weekday')['delivery_time_deviation'].mean()
    plt.figure(figsize=(8, 4))
    
    sns.barplot(
    x=weekday_delay.index,
    y=weekday_delay.values,
    hue=weekday_delay.values,  # use values to color bars
    palette='crest',
    legend=False               # hide legend
    )

    plt.title(" Average Delivery Deviation by Weekday")
    plt.xlabel("Weekday")
    plt.ylabel("Avg Delivery Deviation")
    plt.xticks(rotation=45)
    plt.show()

print("\n Phase 2: EDA completed successfully.")


In [ ]:
# ====================================================
# Phase 5: Machine Learning Modeling
# ====================================================
# 5/0 FEATURE SELECTION & ENCODING

# Drop non-numeric / irrelevant columns
exclude_cols = ['timestamp', 'Customer City', 'Customer Email', 'Customer Fname', 'Customer Lname', 'vehicke_city', 'traffic_category']
m_df = m_df.drop(columns=[c for c in exclude_cols if c in df.columns], errors='ignore')

# Encode categorical variables
for col in m_df.select_dtypes(include='object').columns:
    m_df[col] = LabelEncoder().fit_transform(m_df[col].astype(str))

# Replace remaining NaNs with median
m_df = m_df.fillna(df.median(numeric_only=True))

In [ ]:
# --- 5.1 DATA CLEANUP BEFORE MODELING

# Drop identifier or irrelevant columns if they exist
drop_cols = [
    'timestamp', 'Customer City', 'Customer Email', 
    'Customer Fname', 'Customer Lname', 'vehicle_city', 'weekday'
]
m_df = m_df.drop(columns=[c for c in drop_cols if c in m_df.columns], errors='ignore')

# --- 1. Convert booleans to int
bool_cols = m_df.select_dtypes(include=['bool']).columns
if len(bool_cols):
    m_df[bool_cols] = m_df[bool_cols].astype(int)

# --- 2. Encode categoricals (object / category)
cat_cols = m_df.select_dtypes(include=['object', 'category']).columns
if len(cat_cols):
    print("Encoding categorical columns:", list(cat_cols))
    for col in cat_cols:
        m_df[col] = m_df[col].astype(str).astype('category').cat.codes

# --- 3. Replace any infinite or missing values
m_df = m_df.replace([np.inf, -np.inf], np.nan)
m_df = m_df.fillna(m_df.median(numeric_only=True))

# --- 4. Enforce numeric dtype globally
# m_df = m_df.apply(pd.to_numeric, errors='coerce').fillna(0)


# --- 5. Verify everything is numeric
non_numeric = m_df.select_dtypes(exclude=[np.number]).columns.tolist()
if non_numeric:
    print(" Still non-numeric columns:", non_numeric)
else:
    print(" All columns successfully converted to numeric.")


In [ ]:
# --- 5.2 checking df

# checking dataframe
print("m_df shape:", m_df.shape)
print("m_df columns:", m_df.columns.tolist())


In [ ]:
# --- 5/3 Regression Modeling ---

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tqdm.notebook import tqdm

# --- Target column ---
target_reg = "delivery_time_deviation"

# --- Copy numeric features ---
numeric_cols = m_df.select_dtypes(include=[np.number]).columns.tolist()
# Remove the target column
if 'delivery_time_deviation' in numeric_cols:
    numeric_cols.remove('delivery_time_deviation')
X = m_df[numeric_cols].copy()
y_reg = m_df[target_reg]

# --- 1. Log-transform skewed features ---
for col in ['fuel_consumption_rate', 'lead_time_days']:
    if col in X.columns:
        X[col] = np.log1p(X[col])  # safe for 0 values

# --- 2. Cap outliers in fuel_consumption_rate ---
if 'fuel_consumption_rate' in X.columns:
    Q1 = X['fuel_consumption_rate'].quantile(0.25)
    Q3 = X['fuel_consumption_rate'].quantile(0.75)
    IQR = Q3 - Q1
    lower_whisker = Q1 - 1.5 * IQR
    upper_whisker = Q3 + 1.5 * IQR
    X['fuel_consumption_rate'] = np.where(
        X['fuel_consumption_rate'] > upper_whisker, upper_whisker,
        np.where(X['fuel_consumption_rate'] < lower_whisker, lower_whisker, X['fuel_consumption_rate'])
    )

# --- 3. Impute missing values ---
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)

# --- 4. Scale numeric features ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# --- 5. Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_reg, test_size=0.2, random_state=42
)

print(f"Training samples: {X_train.shape[0]}, Test samples: {X_test.shape[0]}")

# --- 6. Define regression models ---
models_reg = {
    "Linear Regression": LinearRegression(),
    "Random Forest Regressor": RandomForestRegressor(
        n_estimators=50, max_depth=10, n_jobs=-1, random_state=42
    )
}

# --- Train models and evaluate ---
results_reg = {}
print("Training regression models...")
for name in tqdm(models_reg.keys(), desc="Training Progress", leave=False):
    model = models_reg[name]
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    
    mae = mean_absolute_error(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    r2 = r2_score(y_test, preds)
    
    results_reg[name] = {"MAE": mae, "RMSE": rmse, "R²": r2}
    print(f"\n{name} done.")
    print(f"MAE: {mae:.3f} | RMSE: {rmse:.3f} | R²: {r2:.3f}")

# --- Display results ---
results_reg_df = pd.DataFrame(results_reg).T
display(results_reg_df.style.background_gradient(cmap="Blues").format("{:.3f}"))


In [ ]:
# ====================================================
# Phase 5/4: Regression Modeling — Delivery Time Deviation
# ====================================================

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# --- 1. Drop rows with missing target ---
target = 'delivery_time_deviation'
df_model = m_df.dropna(subset=[target]).copy()

# --- 2. Define numeric and categorical features ---
numeric_features = [
    'fuel_consumption_rate', 'eta_variation_hours', 'traffic_congestion_level',
    'warehouse_inventory_level', 'loading_unloading_time', 'port_congestion_level',
    'shipping_costs', 'supplier_reliability_score', 'lead_time_days',
    'historical_demand', 'iot_temperature', 'customs_clearance_time',
    'driver_behavior_score', 'fatigue_monitoring_score', 'avg_route_congestion'
]

categorical_features = [
    'handling_equipment_availability', 'order_fulfillment_status',
    'weather_condition_severity', 'cargo_condition_status',
    'route_risk_level', 'day_of_week', 'month', 'route_cluster',
    'cluster_desc', 'year'
]

# --- 3. Exclude leaky or irrelevant features ---
exclude_features = [
    'disruption_likelihood_score', 'delay_probability', 'risk_classification',
    'eta_minus_actual_hours', 'eta_variation_minutes', 'vehicle_gps_latitude',
    'vehicle_gps_longitude', 'vehicle_location', 'row_id'
]
numeric_features = [f for f in numeric_features if f in df_model.columns and f not in exclude_features]
categorical_features = [f for f in categorical_features if f in df_model.columns and f not in exclude_features]

# --- 4. Prepare X and y safely ---
X = df_model[numeric_features + categorical_features].copy()
y = df_model[target].copy()
assert len(X) == len(y), f"Length mismatch: {len(X)} vs {len(y)}"

print(f"Using {len(numeric_features)} numeric and {len(categorical_features)} categorical features")

# --- 5. Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# --- 6. Preprocessing pipelines ---
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# --- 7. Full pipeline with Random Forest Regressor ---
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(
        n_estimators=100, max_depth=10, random_state=42, n_jobs=-1
    ))
])

# --- 8. Train model ---
pipeline.fit(X_train, y_train)

# --- 9. Evaluate model ---
y_pred = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\nDelivery Time Deviation Prediction Performance:")
print(f"MAE: {mae:.3f}")
print(f"RMSE: {rmse:.3f}")
print(f"R²: {r2:.3f}")

# --- 10. Feature importance safely ---
regressor = pipeline.named_steps['regressor']
preprocessor = pipeline.named_steps['preprocessor']

# Numeric feature names
numeric_names = numeric_features

# Categorical feature names after one-hot encoding
cat_names = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_features)

# Combine
all_features = list(numeric_names) + list(cat_names)

# Match lengths with actual importances
importances = regressor.feature_importances_
feat_imp = pd.Series(importances, index=all_features[:len(importances)]).sort_values(ascending=False)

# --- 11. Plot top 10 feature importances ---
plt.figure(figsize=(10,6))
feat_imp.head(10).plot(kind='barh', color='steelblue')
plt.title("Top 10 Feature Importances — Delivery Time Deviation")
plt.xlabel("Importance")
plt.ylabel("Feature")

